In [3]:
import torch
import torch.nn as nn
cfg = [64, 64, 'M', 128, 128, 'M', 256, 256, 256, 'C', 512, 512, 512, 'M',512, 512, 512]#vgg16:D
def vgg():
    in_channels=3
    layers=[]
    for v in cfg:
        if v=='M':
            layers+=[nn.MaxPool2d(kernel_size=2,stride=2)]
        elif v=='C':
            layers+=[nn.MaxPool2d(kernel_size=2,stride=2,ceil_mode=True)]
        else:
            c=nn.Conv2d(in_channels,v,kernel_size=3,padding=1)
            layers+=[c,nn.ReLU(inplace=True)]
            in_channels=v
    pool5=nn.MaxPool2d(kernel_size=3,stride=1,padding=1)#512*19*19
    #19+6*2=31--->31*31--->13*13--->31-13=18,18/1+1=19,--->1024*19*19
    conv6=nn.Conv2d(512,1024,kernel_size=3,padding=6,dilation=6)#
    conv7=nn.Conv2d(1024,1024,kernel_size=1)#19*19*1024
    layers+=[pool5,conv6,nn.ReLU(),conv7,nn.ReLU()]
    return nn.ModuleList(layers)

In [4]:
vgg()

ModuleList(
  (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (1): ReLU(inplace=True)
  (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (3): ReLU(inplace=True)
  (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (6): ReLU(inplace=True)
  (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (8): ReLU(inplace=True)
  (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (11): ReLU(inplace=True)
  (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (13): ReLU(inplace=True)
  (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (15): ReLU(inplace=True)
  (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=True)
  (17): Conv2d(256, 512, kernel_siz

In [11]:
nn.Conv2d?